In [1]:
import numpy as np
import networkx as nx
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
n=5 #Prime router number
N=10 #regular router number
m=3 #shortest path number
W=1000000

In [3]:
#file_name='full_graph_{}_{}.txt'.format(n,N)
#file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
#file_best_clustering='full_graph_best_clustering_{}_{}.txt'.format(n,N)

file_name_shortest_path='full_graph_shortest_path_{}_{}.txt'.format(n,N)
file_name_prime_routers_shortest_path='prime_graph_shortest_path_{}_{}.txt'.format(n,N)
file_name_prime_graph='prime_graph_{}_{}.txt'.format(n,N)
file_name_prime_graph_optimal_path='prime_graph_optimal_path_{}_{}.txt'.format(n,N)

# Reading graph

In [4]:
con=[[] for _ in range(n+1)]
edges=set()

In [5]:
with open('test/'+file_name_prime_graph, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    #f.write('\n')
    for line in f:
        #print('line',line)
        x,y,w=map(float,line.split())
        x,y=int(x),int(y)
        #print(x,y,w)
        

            
        con[x].append((y,w))
        #con[y].append((x,w))
        edges.add((x,y,w))

           
    f.close()

In [6]:
#con
edges

{(1, 2, 6.089308538078741),
 (2, 1, 6.089308538078741),
 (2, 3, 3.0),
 (2, 4, 4.977825363992857),
 (2, 5, 5.3342404165033095),
 (3, 2, 3.0),
 (3, 5, 3.1448932613183276),
 (4, 2, 4.977825363992857),
 (5, 2, 5.3342404165033095),
 (5, 3, 3.1448932613183276)}

In [7]:
con

[[],
 [(2, 6.089308538078741)],
 [(1, 6.089308538078741),
  (3, 3.0),
  (4, 4.977825363992857),
  (5, 5.3342404165033095)],
 [(2, 3.0), (5, 3.1448932613183276)],
 [(2, 4.977825363992857)],
 [(2, 5.3342404165033095), (3, 3.1448932613183276)]]

# Reading shortest paths

In [8]:
sh_path=[[] for _ in range(n+1)]

for i in range(n+1):
    sh_path[i]=[[] for _ in range(n+1)]

In [9]:
with open('test/'+file_name_prime_routers_shortest_path, 'r') as f:
    
    n=int(f.readline())
    next(f)
    
    for i in range(1,n+1):
        for j in range(1,n+1):
            if i!=j:
                x,y=map(int,f.readline().split())
                #print('x,y',x,y)
                
                while True:
                    line1=f.readline()
                    #print('line1',line1)
                    
                    if not line1.strip():
                        break

                    line2=f.readline()
                    
                    w=float(line1)
                    path=list(map(int,line2.split()))
                    #print('path',path)
                    #print('w',w)

                    sh_path[i][j].append((w,path))


In [10]:
sh_path

[[[], [], [], [], [], []],
 [[],
  [],
  [(6.089308538078741, [1, 2])],
  [(9.08930853807874, [1, 2, 3]), (14.568442215900376, [1, 2, 5, 3])],
  [(11.067133902071598, [1, 2, 4])],
  [(11.42354895458205, [1, 2, 5]), (12.234201799397068, [1, 2, 3, 5])]],
 [[],
  [(6.089308538078741, [2, 1])],
  [],
  [(3.0, [2, 3]), (8.479133677821636, [2, 5, 3])],
  [(4.977825363992857, [2, 4])],
  [(5.3342404165033095, [2, 5]), (6.144893261318328, [2, 3, 5])]],
 [[],
  [(9.08930853807874, [3, 2, 1]), (14.568442215900376, [3, 5, 2, 1])],
  [(3.0, [3, 2]), (8.479133677821636, [3, 5, 2])],
  [],
  [(7.977825363992857, [3, 2, 4]), (13.456959041814493, [3, 5, 2, 4])],
  [(3.1448932613183276, [3, 5]), (8.33424041650331, [3, 2, 5])]],
 [[],
  [(11.067133902071598, [4, 2, 1])],
  [(4.977825363992857, [4, 2])],
  [(7.977825363992857, [4, 2, 3]), (13.456959041814493, [4, 2, 5, 3])],
  [],
  [(10.312065780496166, [4, 2, 5]), (11.122718625311185, [4, 2, 3, 5])]],
 [[],
  [(11.42354895458205, [5, 2, 1]), (12.234201

# Figuring out the coefficients alpha

In [11]:
alpha=set()

In [12]:
def belong_to_path():
   
    for i in range(n+1):
        for j in range(n+1):
            if i!=j:
                for s in range(len(sh_path[i][j])):
                    path=sh_path[i][j][s][1]
                    l=len(path)
                    
                    for k in range(l-1):
                        alpha.add((path[k],path[k+1],s,i,j))

In [13]:
belong_to_path()

In [14]:
alpha

{(1, 2, 0, 1, 2),
 (1, 2, 0, 1, 3),
 (1, 2, 0, 1, 4),
 (1, 2, 0, 1, 5),
 (1, 2, 1, 1, 3),
 (1, 2, 1, 1, 5),
 (2, 1, 0, 2, 1),
 (2, 1, 0, 3, 1),
 (2, 1, 0, 4, 1),
 (2, 1, 0, 5, 1),
 (2, 1, 1, 3, 1),
 (2, 1, 1, 5, 1),
 (2, 3, 0, 1, 3),
 (2, 3, 0, 2, 3),
 (2, 3, 0, 4, 3),
 (2, 3, 1, 1, 5),
 (2, 3, 1, 2, 5),
 (2, 3, 1, 4, 5),
 (2, 3, 1, 5, 3),
 (2, 4, 0, 1, 4),
 (2, 4, 0, 2, 4),
 (2, 4, 0, 3, 4),
 (2, 4, 0, 5, 4),
 (2, 4, 1, 3, 4),
 (2, 4, 1, 5, 4),
 (2, 5, 0, 1, 5),
 (2, 5, 0, 2, 5),
 (2, 5, 0, 4, 5),
 (2, 5, 1, 1, 3),
 (2, 5, 1, 2, 3),
 (2, 5, 1, 3, 5),
 (2, 5, 1, 4, 3),
 (3, 2, 0, 3, 1),
 (3, 2, 0, 3, 2),
 (3, 2, 0, 3, 4),
 (3, 2, 1, 3, 5),
 (3, 2, 1, 5, 1),
 (3, 2, 1, 5, 2),
 (3, 2, 1, 5, 4),
 (3, 5, 0, 3, 5),
 (3, 5, 1, 1, 5),
 (3, 5, 1, 2, 5),
 (3, 5, 1, 3, 1),
 (3, 5, 1, 3, 2),
 (3, 5, 1, 3, 4),
 (3, 5, 1, 4, 5),
 (4, 2, 0, 4, 1),
 (4, 2, 0, 4, 2),
 (4, 2, 0, 4, 3),
 (4, 2, 0, 4, 5),
 (4, 2, 1, 4, 3),
 (4, 2, 1, 4, 5),
 (5, 2, 0, 5, 1),
 (5, 2, 0, 5, 2),
 (5, 2, 0, 5, 4),
 (5, 2, 1,

# Collecting terms

In [15]:
def mult(a):
    d={}
    
    for q in a:
        for e in q['term']:
            for e2 in q['term']:
                
                x,y=e
                z,t=e2
                
                if x<z:
                    x,z=z,x
                    y,t=t,y
                
                coeff=q['coeff']
                            
                if x!='C' and z!='C':
                    d[(x,z)]=d[(x,z)]+coeff*y*t if (x,z) in d else coeff*y*t
                    
                if x=='C' and z!='C':
                    d[(z,z)]=d[(z,z)]+coeff*y*t if (z,z) in d else coeff*y*t
                    
                if x!='C' and z=='C':
                    d[(x,x)]=d[(x,x)]+coeff*y*t if (x,x) in d else coeff*y*t
                    
        #print(d)
        

                    
    return d

# Example

In [16]:
form=[{'coeff':1, 'term':[('y2',1)]}, {'coeff':4, 'term':[('x1',1), ('x2',1), ('y2',1)]}, {'coeff':1, 'term':[('x2',1)]}, {'coeff':1, 'term':[('y1',1)]}, 
      {'coeff':1, 'term':[('x1',1), ('y2',1)]}, {'coeff':9, 'term':[('x1',1), ('y1',1)]}, {'coeff':25, 'term':[('x1',1), ('x2',1), ('C',-1)]}, {'coeff':25, 'term':[('y1',1), ('y2',1), ('C',-1)]}]

In [17]:
Q=mult(form) #we ignore a constant term

In [18]:
Q #we are ignoring the constant term and using the fact that x^2=x, x in {0,1}

{('y2', 'y2'): -19,
 ('x1', 'x1'): -11,
 ('x2', 'x1'): 58,
 ('y2', 'x1'): 10,
 ('x2', 'x2'): -20,
 ('y2', 'x2'): 8,
 ('y1', 'y1'): -15,
 ('y1', 'x1'): 18,
 ('y2', 'y1'): 50}

# Real network example

In [19]:
form=[0]*(n+1)

for i in range(n+1):
    form[i]=[0]*(n+1)

In [20]:
D=np.random.rand(n+1,n+1)

In [21]:
form=[]
xs=set()

In [22]:
for e in edges:
    x,y,w=e
    term=[]
    coeff=0
    
    for i in range(n+1):
        for j in range(n+1):
            if i!=j:
                
                for s in range(len(sh_path[i][j])):
                    
                    if (x,y,s,i,j) in alpha:
                        coeff=1.0/w
                        name='x'+'_'+str(s)+'_'+str(i)+'_'+str(j)
                        term.append((name,D[i][j]))
                        xs.add(name)
            
            
                if coeff!=0:
                    form.append({'coeff':coeff, 'term':term})                

In [23]:
len(xs)

34

In [24]:
edges

{(1, 2, 6.089308538078741),
 (2, 1, 6.089308538078741),
 (2, 3, 3.0),
 (2, 4, 4.977825363992857),
 (2, 5, 5.3342404165033095),
 (3, 2, 3.0),
 (3, 5, 3.1448932613183276),
 (4, 2, 4.977825363992857),
 (5, 2, 5.3342404165033095),
 (5, 3, 3.1448932613183276)}

In [25]:
xs

{'x_0_1_2',
 'x_0_1_3',
 'x_0_1_4',
 'x_0_1_5',
 'x_0_2_1',
 'x_0_2_3',
 'x_0_2_4',
 'x_0_2_5',
 'x_0_3_1',
 'x_0_3_2',
 'x_0_3_4',
 'x_0_3_5',
 'x_0_4_1',
 'x_0_4_2',
 'x_0_4_3',
 'x_0_4_5',
 'x_0_5_1',
 'x_0_5_2',
 'x_0_5_3',
 'x_0_5_4',
 'x_1_1_3',
 'x_1_1_5',
 'x_1_2_3',
 'x_1_2_5',
 'x_1_3_1',
 'x_1_3_2',
 'x_1_3_4',
 'x_1_3_5',
 'x_1_4_3',
 'x_1_4_5',
 'x_1_5_1',
 'x_1_5_2',
 'x_1_5_3',
 'x_1_5_4'}

In [26]:
for i in range(n+1):
    for j in range(n+1):
        if i!=j:

            #coeff=0
            term=[]

            for s in range(len(sh_path[i][j])):
                name='x'+'_'+str(s)+'_'+str(i)+'_'+str(j)

                if name in xs:
                    term.append((name,1))


            term.append(('C',-1))

            #print('term',term)

            #if coeff!=0:
            form.append({'coeff':W, 'term':term})   
                        
                    
                    

In [27]:
form

[{'coeff': 0.16422225836424995,
  'term': [('x_0_1_2', 0.43994379345756296),
   ('x_0_1_3', 0.6438220261005577),
   ('x_1_1_3', 0.6438220261005577),
   ('x_0_1_4', 0.7990475464705086),
   ('x_0_1_5', 0.4073918289024515),
   ('x_1_1_5', 0.4073918289024515)]},
 {'coeff': 0.16422225836424995,
  'term': [('x_0_1_2', 0.43994379345756296),
   ('x_0_1_3', 0.6438220261005577),
   ('x_1_1_3', 0.6438220261005577),
   ('x_0_1_4', 0.7990475464705086),
   ('x_0_1_5', 0.4073918289024515),
   ('x_1_1_5', 0.4073918289024515)]},
 {'coeff': 0.16422225836424995,
  'term': [('x_0_1_2', 0.43994379345756296),
   ('x_0_1_3', 0.6438220261005577),
   ('x_1_1_3', 0.6438220261005577),
   ('x_0_1_4', 0.7990475464705086),
   ('x_0_1_5', 0.4073918289024515),
   ('x_1_1_5', 0.4073918289024515)]},
 {'coeff': 0.16422225836424995,
  'term': [('x_0_1_2', 0.43994379345756296),
   ('x_0_1_3', 0.6438220261005577),
   ('x_1_1_3', 0.6438220261005577),
   ('x_0_1_4', 0.7990475464705086),
   ('x_0_1_5', 0.4073918289024515),
  

In [28]:
Q=mult(form)

In [29]:
Q

{('x_0_1_2', 'x_0_1_2'): -999999.2371526316,
 ('x_0_1_3', 'x_0_1_2'): 2.232730387981277,
 ('x_1_1_3', 'x_0_1_2'): 2.232730387981277,
 ('x_0_1_4', 'x_0_1_2'): 2.771041788135307,
 ('x_0_1_5', 'x_0_1_2'): 1.4128067685334278,
 ('x_1_1_5', 'x_0_1_2'): 1.4128067685334278,
 ('x_0_1_3', 'x_0_1_3'): -999995.188404691,
 ('x_1_1_3', 'x_0_1_3'): 2000003.2674196647,
 ('x_0_1_4', 'x_0_1_3'): 4.055194697544189,
 ('x_0_1_5', 'x_0_1_3'): 2.067528010924222,
 ('x_1_1_5', 'x_0_1_3'): 6.0892747790029755,
 ('x_1_1_3', 'x_1_1_3'): -999993.5475613805,
 ('x_1_1_3', 'x_0_1_4'): 4.055194697544189,
 ('x_1_1_3', 'x_0_1_5'): 4.3293758771936695,
 ('x_1_1_5', 'x_1_1_3'): 2.067528010924222,
 ('x_0_1_4', 'x_0_1_4'): -999994.6617355814,
 ('x_0_1_5', 'x_0_1_4'): 2.5660091103034413,
 ('x_1_1_5', 'x_0_1_4'): 2.5660091103034413,
 ('x_0_1_5', 'x_0_1_5'): -999998.6302485119,
 ('x_1_1_5', 'x_0_1_5'): 2000001.3082715152,
 ('x_1_1_5', 'x_1_1_5'): -999996.9651914285,
 ('x_0_2_3', 'x_0_1_3'): 6.567282454564978,
 ('x_1_2_5', 'x_0_1

In [30]:
len(Q)

206

In [31]:
sampler = EmbeddingComposite(DWaveSampler())

In [32]:
response=sampler.sample_qubo(Q, num_reads=1)

In [33]:
for data in response.data(['sample','energy', 'num_occurrences']):
    
    print(data.sample, 'energy', data.energy, 'num_occurrences', data.num_occurrences) 

{'x_0_1_2': 1, 'x_0_1_3': 1, 'x_0_1_4': 1, 'x_0_1_5': 0, 'x_0_2_1': 1, 'x_0_2_3': 1, 'x_0_2_4': 1, 'x_0_2_5': 0, 'x_0_3_1': 0, 'x_0_3_2': 1, 'x_0_3_4': 0, 'x_0_3_5': 0, 'x_0_4_1': 1, 'x_0_4_2': 1, 'x_0_4_3': 0, 'x_0_4_5': 0, 'x_0_5_1': 1, 'x_0_5_2': 0, 'x_0_5_3': 1, 'x_0_5_4': 0, 'x_1_1_3': 0, 'x_1_1_5': 1, 'x_1_2_3': 0, 'x_1_2_5': 1, 'x_1_3_1': 1, 'x_1_3_2': 0, 'x_1_3_4': 1, 'x_1_3_5': 1, 'x_1_4_3': 1, 'x_1_4_5': 1, 'x_1_5_1': 0, 'x_1_5_2': 1, 'x_1_5_3': 0, 'x_1_5_4': 1} energy -19999759.591255344 num_occurrences 1


# Saving optimal trajectories

In [39]:
with open('test/'+file_name_prime_graph_optimal_path, 'w') as f:
    
    f.write(str(n)+'\n\n')

    for x in data.sample:
        if data.sample[x]:
            #print('x',x)

            s,i,j=map(int,x.split('_')[1:])
            
            #print('s,i,j',s,i,j)
            
            f.write('{} {}\n'.format(i,j))
            
            #print('sh_path[i][j][s][1]',sh_path[i][j][s][1])
            f.write(' '.join(map(str,sh_path[i][j][s][1]))+'\n\n')
    
    